In [75]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime
import io

pd.options.display.max_colwidth = 1000000000

my_headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36",
              "Accept":"text/html,application/xhtml+xml,application/xml; q=0.9,image/webp,image/apng,*/*;q=0.8",
              "Accept-Language":"en-US,en;q=0.9,en-GB;q=0.8", "Content-Language":"en-US,en;q=0.9,en-GB;q=0.8"}

### RaceInfo:

In [76]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# set up driver (check for copatable chrome version: https://chromedriver.chromium.org/downloads):
driver = webdriver.Chrome()
driver.implicitly_wait(30)

driver.get('https://racing.hkjc.com/racing/information/English/racing/RaceCard.aspx')
#driver.get('https://racing.hkjc.com/racing/information/English/racing/RaceCard.aspx?RaceDate=2021/12/18&Racecourse=ST&RaceNo=10')

raceInfoTab = driver.find_element_by_class_name("margin_top10")
#raceInfoTabHTML = infoTab.get_attribute('outerHTML')
#raceInfo_soup = BeautifulSoup(raceInfoTabHTML, 'lxml')
#bn_raceInfo = bn_soup.find_all('div',{'class':'f_fs13'})[1].contents

<ipython-input-76-e2904d9f59f7>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  raceInfoTab = driver.find_element_by_class_name("margin_top10")


Race info ideally need:
- Date
- Course
- Meeting Number (race number today eg.1-10 @ Sha Tin)
- Class
- Distance
- Rating (aka Ranking)
- Race_Type
- Surface
- Prize (in HKD)
- Runners (Count Horses)
- Going

In [77]:
raceInfoTabHTML = raceInfoTab.get_attribute('outerHTML')
raceInfo_soup = BeautifulSoup(raceInfoTabHTML, 'lxml')

In [78]:
bn_raceInfo = raceInfo_soup.find('div',{'class':'f_fs13'})
bn_raceInfo1 = raceInfo_soup.find('span',{'class':'font_wb'})
bn_raceInfo2 = raceInfo_soup.find('span',{'class':'br/'})
bn_raceInfo3 = raceInfo_soup.find_all('span',{'class':'font_wb'})[0].contents

bn_raceInfo3 = raceInfo_soup.get_text(strip=True, separator= " ")
#print(raceInfo_soup.get_text(strip=True, separator=" "))

print(bn_raceInfo3)

Race 10 - LUKFOOK JEWELLERY F-STYLE HEY COOL COLLECTION HANDICAP Saturday, December 18, 2021, Sha Tin, 17:45 Turf, "C+3" Course, 1600M, Good Prize Money: $1,570,000, Rating: 80-60, Class 3 Raceday Focus


In [79]:
#bn_raceInfo4 = bn_raceInfo3.split(",")
bn_raceInfo4 = bn_raceInfo3.replace(' - ', ', ').replace('Turf,', 'Turf').split(', ')
#bn_raceInfo4 = bn_raceInfo3.split(', ')

print(bn_raceInfo4)

['Race 10', 'LUKFOOK JEWELLERY F-STYLE HEY COOL COLLECTION HANDICAP Saturday', 'December 18', '2021', 'Sha Tin', '17:45 Turf "C+3" Course', '1600M', 'Good Prize Money: $1,570,000', 'Rating: 80-60', 'Class 3 Raceday Focus']


In [80]:
bn_raceInfo5 = pd.DataFrame({'col': bn_raceInfo4}).transpose()
bn_raceInfo5["Date"] = (bn_raceInfo5[2] + ' ' + bn_raceInfo5[3]) # Race date string
bn_raceInfo5["Date"] =  pd.to_datetime(bn_raceInfo5["Date"], format = '%B %d %Y').dt.strftime('%m/%d/%Y') # Race date string to date & format
#bn_raceInfo5["Date"] = bn_raceInfo5["Date"].dt.strftime('%m/%d/%Y') # Race date format
bn_raceInfo5[0] = bn_raceInfo5[0].str.lstrip('Race ') # Meeting_Number
bn_raceInfo5[1] = bn_raceInfo5[1].str.rsplit(' ', n=1).str.get(0) # Race Type
bn_raceInfo5[6] = bn_raceInfo5[6].str.rstrip('M') # Distance
bn_raceInfo5[9] = bn_raceInfo5[9].str.rstrip(' Raceday Focus') # Class
bn_raceInfo5[8] = bn_raceInfo5[8].str.lstrip('Rating: ') # Ranking
bn_raceInfo5[7] = bn_raceInfo5[7].str.lstrip('Good Prize Money: $').str.replace(r',', '') # Prize
bn_raceInfo5["Surface"] = bn_raceInfo5[5].str.split(' ', 1).str.get(1).str.replace(r'"', '').str.upper() # Surface

bn_raceInfo5 = bn_raceInfo5.drop(bn_raceInfo5.columns[[2,3,5]], axis=1) # drop uneeded columns
bn_raceInfo5 = bn_raceInfo5.rename(columns={0:'Meeting_Number',1:'Race_Type',4:'Course',
                                           6:'Distance',7:'Prize',8:'Ranking',9:'Class'}) # rename final columns
raceInfo = bn_raceInfo5.reset_index(drop=True)

raceInfo.head()

,Meeting_Number,Race_Type,Course,Distance,Prize,Ranking,Class,Date,Surface
0,10,LUKFOOK JEWELLERY F-STYLE HEY COOL COLLECTION HANDICAP,Sha Tin,1600,1570000,80-60,Class 3,12/18/2021,TURF C+3 COURSE


### RaceCard:

In [81]:
raceCard = pd.read_html(driver.find_element_by_id("racecardlist").get_attribute('outerHTML'))[1]

<ipython-input-81-444b7c7ed60d>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  raceCard = pd.read_html(driver.find_element_by_id("racecardlist").get_attribute('outerHTML'))[1]


In [82]:
columns_to_drop = (['Horse No.', 'Last 6 Runs', 'Colour', 'Rtg.+/-','Priority','Gear'])

raceCard['Jockey'] = raceCard['Jockey'].str.split('(').str[0]
raceCard['Trainer'] = raceCard['Trainer'].str.split('(').str[0]
raceCard.drop(columns_to_drop, axis=1, inplace=True)
raceCard = raceCard.rename({'Wt.': 'Weight', 'Horse Wt. (Declaration)': 'Weight_Declared'}, axis=1)

# rename
# Wt. = 'Weight'
# Horse Wt. (Declaration) = 'Weight_Declared'


#df.raceCard()

#raceCard['Weight'] = raceCard.['Horse Wt. (Declaration)'].combine_first(raceCard.['Wt.'])

#raceCard['Weight'] = np.where(raceCard["Horse Wt. (Declaration)"].isnull(), raceCard["Wt."], raceCard["Horse Wt. (Declaration)"] )

raceCard.head()

,Horse,Weight,Jockey,Draw,Trainer,Rtg.,Weight_Declared
0,LOOKING GREAT,133,K C Leung,13,F C Lor,80,1065
1,CAPTAIN WIN,130,C L Chau,14,D J Hall,77,1229
2,CRAIG'S STAR,126,H Bentley,6,K H Ting,73,1260
3,SUPER TEN,126,A Badel,10,D J Whyte,73,1233
4,INTREPID WINNER,125,A Hamelin,8,A T Millard,72,1054


### RaceDay Weather Info:

Weather:
- Min Temp.
- Max Temp.
- Humidity

In [83]:
# Day of Race:
Date_of_race = pd.to_datetime(bn_raceInfo5["Date"], format = '%m/%d/%Y').dt.strftime('%d %b').iloc[0] + ' '
Day_of_race = pd.to_datetime(bn_raceInfo5["Date"], format = '%m/%d/%Y').dt.strftime('%A').iloc[0]
location = (bn_raceInfo5["Course"].str.replace(r' ', '')).iloc[0]

print(Day_of_race)
print(location)

user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
language = "en-US,en;q=0.9,en-GB;q=0.8"

weather_url = "https://www.google.com/search?q=weather+"+location
print(weather_url)

Saturday
ShaTin
https://www.google.com/search?q=weather+ShaTin


In [84]:
def get_weather_data(weather_url):
    session = requests.Session()
    session.headers['User-Agent'] = user_agent
    session.headers['Accept-Language'] = language
    session.headers['Content-Language'] = language
    html = session.get(weather_url)
    # create a new soup
    soup = BeautifulSoup(html.text, "html.parser")
    # store all results on this dictionary
    result = {}
    # extract region
    result['region'] = soup.find("div", attrs={"id": "wob_loc"}).text
    # extract temperature now
    result['temp_now'] = soup.find("span", attrs={"id": "wob_tm"}).text
    # get the day and hour now
    result['dayhour'] = soup.find("div", attrs={"id": "wob_dts"}).text
    # get the actual weather
    result['weather_now'] = soup.find("span", attrs={"id": "wob_dc"}).text
    # get the precipitation
    result['precipitation'] = soup.find("span", attrs={"id": "wob_pp"}).text
    # get the % of humidity
    result['humidity'] = soup.find("span", attrs={"id": "wob_hm"}).text
    # extract the wind
    result['wind'] = soup.find("span", attrs={"id": "wob_ws"}).text
    # get next few days' weather
    next_days = []
    days = soup.find("div", attrs={"id": "wob_dp"})
    for day in days.findAll("div", attrs={"class": "wob_df"}):
        # extract the name of the day
        day_name = day.findAll("div")[0].attrs['aria-label']
        # get weather status for that day
        weather = day.find("img").attrs["alt"]
        temp = day.findAll("span", {"class": "wob_t"})
        # maximum temparature in Celsius, use temp[1].text if you want fahrenheit
        max_temp = temp[0].text
        # minimum temparature in Celsius, use temp[3].text if you want fahrenheit
        min_temp = temp[2].text
        next_days.append({"name": day_name, "weather": weather, "max_temp": max_temp, "min_temp": min_temp})
    # append to result
    result['next_days'] = next_days
    return result

In [85]:
if __name__ == "__main__":
    URL = weather_url
    # get data
    data = get_weather_data(URL)
    # print data
    for dayweather in data["next_days"]:
        #print("="*40, dayweather["name"], "="*40)
        print(dayweather["name"]+":")
        #print("Description:", dayweather["weather"])
        print(f"Max temperature: {dayweather['max_temp']}°C")
        print(f"Min temperature: {dayweather['min_temp']}°C")
        if dayweather["name"] == Day_of_race:
            break

Saturday:
Max temperature: 21°C
Min temperature: 14°C


In [86]:
# RaceDay Min - Max Temp:

min_max_temp = io.StringIO(dayweather['min_temp'] + ' ' + dayweather['max_temp'])
weatherInfo = pd.read_csv(min_max_temp, sep=" ", header = None)
weatherInfo.rename({0: 'Min_Temp', 1: 'Max_Temp'}, axis=1, inplace=True)

weatherInfo.head()

,Min_Temp,Max_Temp
0,14,21


### Cross Join RaceCard with RaceInfo & WeatherInfo:

In [87]:
# Info on Specific Race:

Race_Info = bn_raceInfo5.iloc[0]
#print(Race_Info)

Race_Title = (raceInfo['Date'].iloc[0] + '-'
              + raceInfo['Course'].iloc[0] + '-'
              + raceInfo['Meeting_Number'].iloc[0]).replace(r'/', '').replace(r' ', '_')

print(Race_Title)

12182021-Sha_Tin-10


In [88]:
raceCard['key'] = 0
raceInfo['key'] = 0
weatherInfo['key'] = 0

raceCardInfo = raceCard.merge(raceInfo, on = 'key', how = 'outer')
raceCardInfo = raceCardInfo.merge(weatherInfo, on = 'key', how = 'outer')
raceCardInfo = raceCardInfo.drop('key', 1)

#print(raceCardInfo)
raceCardInfo.head()

,Horse,Weight,Jockey,Draw,Trainer,Rtg.,Weight_Declared,Meeting_Number,Race_Type,Course,Distance,Prize,Ranking,Class,Date,Surface,Min_Temp,Max_Temp
0,LOOKING GREAT,133,K C Leung,13,F C Lor,80,1065,10,LUKFOOK JEWELLERY F-STYLE HEY COOL COLLECTION HANDICAP,Sha Tin,1600,1570000,80-60,Class 3,12/18/2021,TURF C+3 COURSE,14,21
1,CAPTAIN WIN,130,C L Chau,14,D J Hall,77,1229,10,LUKFOOK JEWELLERY F-STYLE HEY COOL COLLECTION HANDICAP,Sha Tin,1600,1570000,80-60,Class 3,12/18/2021,TURF C+3 COURSE,14,21
2,CRAIG'S STAR,126,H Bentley,6,K H Ting,73,1260,10,LUKFOOK JEWELLERY F-STYLE HEY COOL COLLECTION HANDICAP,Sha Tin,1600,1570000,80-60,Class 3,12/18/2021,TURF C+3 COURSE,14,21
3,SUPER TEN,126,A Badel,10,D J Whyte,73,1233,10,LUKFOOK JEWELLERY F-STYLE HEY COOL COLLECTION HANDICAP,Sha Tin,1600,1570000,80-60,Class 3,12/18/2021,TURF C+3 COURSE,14,21
4,INTREPID WINNER,125,A Hamelin,8,A T Millard,72,1054,10,LUKFOOK JEWELLERY F-STYLE HEY COOL COLLECTION HANDICAP,Sha Tin,1600,1570000,80-60,Class 3,12/18/2021,TURF C+3 COURSE,14,21


In [89]:
# Save as CSV:

raceCardInfo.to_csv('raceCardInfo_' + Race_Title + '.csv', index=False)